In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession, DataFrame

In [2]:
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/14 16:40:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/14 16:40:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/14 16:40:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
time_zone = "Europe/Rome" # America/New_York

spark.conf.set('spark.sql.session.timeZone', time_zone)

In [4]:
data_df = spark.createDataFrame([
    (348272371, "2023-01-01", 5.50, "shopping", "true"),
    (348272371, "2023-01-01", 6.10, "salute", "false"),
    (348272371, "2023-01-01", 8.20, "trasporti", "false"),
    (348272371, "2023-01-01", 1.50, "trasporti", "true"),
    (348272371, "2023-01-06", 20.20, "shopping", "false"),
    (348272371, "2023-01-06", 43.00, "shopping", "true"),
    (348272371, "2023-01-06", 72.20, "shopping", "false"),
    (234984832, "2023-01-01", 15.34, "salute", "true"),
    (234984832, "2023-01-01", 36.22, "salute", "true"),
    (234984832, "2023-01-01", 78.35, "salute", "false"),
    (234984832, "2023-01-02", 2.20, "trasporti", "true"),
], schema=["ID_BIC_CLIENTE", "DATA_TRANSAZIONE", "IMPORTO", "CA_CATEGORY_LIV0", "IS_CARTA"])

data_df = data_df.withColumn("DATA_TRANSAZIONE", F.to_timestamp(F.col("DATA_TRANSAZIONE"), "yyyy-MM-dd"))

In [13]:
data_df = spark.createDataFrame([
    (348272371, "2023-01-01", 5.50, "shopping", "true"),
    (348272371, "2023-01-02", 5.50, "shopping", "true"),
    (348272371, "2023-03-26", 6.10, "salute", "false"),
    (348272371, "2023-03-27", 6.10, "salute", "false"),
], schema=["ID_BIC_CLIENTE", "DATA_TRANSAZIONE", "IMPORTO", "CA_CATEGORY_LIV0", "IS_CARTA"])

data_df = data_df.withColumn("DATA_TRANSAZIONE", F.to_timestamp(F.col("DATA_TRANSAZIONE"), "yyyy-MM-dd"))

In [14]:
data_df.printSchema()

root
 |-- ID_BIC_CLIENTE: long (nullable = true)
 |-- DATA_TRANSAZIONE: timestamp (nullable = true)
 |-- IMPORTO: double (nullable = true)
 |-- CA_CATEGORY_LIV0: string (nullable = true)
 |-- IS_CARTA: string (nullable = true)



In [15]:
data_df.show()

+--------------+-------------------+-------+----------------+--------+
|ID_BIC_CLIENTE|   DATA_TRANSAZIONE|IMPORTO|CA_CATEGORY_LIV0|IS_CARTA|
+--------------+-------------------+-------+----------------+--------+
|     348272371|2023-01-01 00:00:00|    5.5|        shopping|    true|
|     348272371|2023-01-02 00:00:00|    5.5|        shopping|    true|
|     348272371|2023-03-26 00:00:00|    6.1|          salute|   false|
|     348272371|2023-03-27 00:00:00|    6.1|          salute|   false|
+--------------+-------------------+-------+----------------+--------+



In [16]:
data_df = data_df.withColumn("DATA_TRANSAZIONE_utc", F.to_utc_timestamp(F.col("DATA_TRANSAZIONE"), time_zone))
data_df.show()

+--------------+-------------------+-------+----------------+--------+--------------------+
|ID_BIC_CLIENTE|   DATA_TRANSAZIONE|IMPORTO|CA_CATEGORY_LIV0|IS_CARTA|DATA_TRANSAZIONE_utc|
+--------------+-------------------+-------+----------------+--------+--------------------+
|     348272371|2023-01-01 00:00:00|    5.5|        shopping|    true| 2022-12-31 23:00:00|
|     348272371|2023-01-02 00:00:00|    5.5|        shopping|    true| 2023-01-01 23:00:00|
|     348272371|2023-03-26 00:00:00|    6.1|          salute|   false| 2023-03-25 23:00:00|
|     348272371|2023-03-27 00:00:00|    6.1|          salute|   false| 2023-03-26 22:00:00|
+--------------+-------------------+-------+----------------+--------+--------------------+



In [21]:
windows_size = "2 days"
utc_shift_hours = 1

#  data_df = data_df.withColumn("window", F.window(timeColumn=F.col("DATA_TRANSAZIONE"), windowDuration=windows_size, startTime=f"-{utc_shift_hours} hours"))
data2_df = data_df.withColumn("window", F.window(timeColumn=F.col("DATA_TRANSAZIONE"), windowDuration=windows_size))
data2_df = data2_df.withColumn("window_start", F.col("window").start)
data2_df = data2_df.withColumn("window_end", F.col("window").end)
data2_df = data2_df.withColumn("window_start_utc", F.to_utc_timestamp(F.col("window_start"), time_zone))
data2_df = data2_df.withColumn("window_end_utc", F.to_utc_timestamp(F.col("window_end"), time_zone))
data2_df.show(truncate=False)

+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|ID_BIC_CLIENTE|DATA_TRANSAZIONE   |IMPORTO|CA_CATEGORY_LIV0|IS_CARTA|DATA_TRANSAZIONE_utc|window                                    |window_start       |window_end         |window_start_utc   |window_end_utc     |
+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|348272371     |2023-01-01 00:00:00|5.5    |shopping        |true    |2022-12-31 23:00:00 |{2022-12-30 01:00:00, 2023-01-01 01:00:00}|2022-12-30 01:00:00|2023-01-01 01:00:00|2022-12-30 00:00:00|2023-01-01 00:00:00|
|348272371     |2023-01-02 00:00:00|5.5    |shopping        |true    |2023-01-01 23:00:00 |{2023-01-01 01:00:00, 2023-01-03 01:00:00}|2023-0

23/07/14 20:16:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 965532 ms exceeds timeout 120000 ms
23/07/14 20:16:03 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/14 20:16:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

In [18]:
spark.conf.set('spark.sql.session.timeZone', "UTC")

windows_size = "2 days"
utc_shift_hours = 1

#  data_df = data_df.withColumn("window", F.window(timeColumn=F.col("DATA_TRANSAZIONE"), windowDuration=windows_size, startTime=f"-{utc_shift_hours} hours"))
data3_df = data_df.withColumn("window", F.window(timeColumn=F.col("DATA_TRANSAZIONE_utc"), windowDuration=windows_size))
data3_df = data3_df.withColumn("window_start", F.col("window").start)
data3_df = data3_df.withColumn("window_end", F.col("window").end)
data3_df = data3_df.withColumn("window_start_utc", F.to_utc_timestamp(F.col("window_start"), time_zone))
data3_df = data3_df.withColumn("window_end_utc", F.to_utc_timestamp(F.col("window_end"), time_zone))
data3_df.show(truncate=False)

spark.conf.set('spark.sql.session.timeZone', time_zone)

+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|ID_BIC_CLIENTE|DATA_TRANSAZIONE   |IMPORTO|CA_CATEGORY_LIV0|IS_CARTA|DATA_TRANSAZIONE_utc|window                                    |window_start       |window_end         |window_start_utc   |window_end_utc     |
+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+-------------------+-------------------+-------------------+-------------------+
|348272371     |2022-12-31 23:00:00|5.5    |shopping        |true    |2022-12-31 22:00:00 |{2022-12-30 00:00:00, 2023-01-01 00:00:00}|2022-12-30 00:00:00|2023-01-01 00:00:00|2022-12-29 23:00:00|2022-12-31 23:00:00|
|348272371     |2023-01-01 23:00:00|5.5    |shopping        |true    |2023-01-01 22:00:00 |{2023-01-01 00:00:00, 2023-01-03 00:00:00}|2023-0

In [19]:
windows_size = "2 days"
utc_shift_hours = 1

data4_df = data_df.withColumn("window", F.window(timeColumn=F.col("DATA_TRANSAZIONE"), windowDuration=windows_size))
# data4_df = data4_df.withColumn("window_start", F.col("window").start)
# data4_df = data4_df.withColumn("window_end", F.col("window").end)
# data4_df = data4_df.withColumn("window_start_utc", F.to_utc_timestamp(F.col("window_start"), time_zone))
# data4_df = data4_df.withColumn("window_end_utc", F.to_utc_timestamp(F.col("window_end"), time_zone))
data4_df.show(truncate=False)

+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+
|ID_BIC_CLIENTE|DATA_TRANSAZIONE   |IMPORTO|CA_CATEGORY_LIV0|IS_CARTA|DATA_TRANSAZIONE_utc|window                                    |
+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+
|348272371     |2023-01-01 00:00:00|5.5    |shopping        |true    |2022-12-31 23:00:00 |{2022-12-30 01:00:00, 2023-01-01 01:00:00}|
|348272371     |2023-01-02 00:00:00|5.5    |shopping        |true    |2023-01-01 23:00:00 |{2023-01-01 01:00:00, 2023-01-03 01:00:00}|
|348272371     |2023-03-26 00:00:00|6.1    |salute          |false   |2023-03-25 23:00:00 |{2023-03-24 01:00:00, 2023-03-26 01:00:00}|
|348272371     |2023-03-27 00:00:00|6.1    |salute          |false   |2023-03-26 22:00:00 |{2023-03-26 01:00:00, 2023-03-28 02:00:00}|
+--------------+-------------------+-------+-----------

In [20]:
windows_size = "2 days"
utc_shift_hours = 1

spark.conf.set('spark.sql.session.timeZone', "UTC")
data5_df = data_df.withColumn("window", F.window(timeColumn=F.col("DATA_TRANSAZIONE_utc"), windowDuration=windows_size))

# data4_df = data4_df.withColumn("window_start", F.col("window").start)
# data4_df = data4_df.withColumn("window_end", F.col("window").end)
# data4_df = data4_df.withColumn("window_start_utc", F.to_utc_timestamp(F.col("window_start"), time_zone))
# data4_df = data4_df.withColumn("window_end_utc", F.to_utc_timestamp(F.col("window_end"), time_zone))
data5_df.show(truncate=False)

spark.conf.set('spark.sql.session.timeZone', time_zone)

+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+
|ID_BIC_CLIENTE|DATA_TRANSAZIONE   |IMPORTO|CA_CATEGORY_LIV0|IS_CARTA|DATA_TRANSAZIONE_utc|window                                    |
+--------------+-------------------+-------+----------------+--------+--------------------+------------------------------------------+
|348272371     |2022-12-31 23:00:00|5.5    |shopping        |true    |2022-12-31 22:00:00 |{2022-12-30 00:00:00, 2023-01-01 00:00:00}|
|348272371     |2023-01-01 23:00:00|5.5    |shopping        |true    |2023-01-01 22:00:00 |{2023-01-01 00:00:00, 2023-01-03 00:00:00}|
|348272371     |2023-03-25 23:00:00|6.1    |salute          |false   |2023-03-25 22:00:00 |{2023-03-24 00:00:00, 2023-03-26 00:00:00}|
|348272371     |2023-03-26 22:00:00|6.1    |salute          |false   |2023-03-26 20:00:00 |{2023-03-26 00:00:00, 2023-03-28 00:00:00}|
+--------------+-------------------+-------+-----------